In [5]:
import pandas as pd

In [6]:
db = pd.read_excel('../data/01_raw/Base_clientes_Monopoly.xlsx')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Juanjo\Desktop\Monopoly\mono\Lib\site-packages\pandas\compat\_optional.py:135 in        │
│ import_optional_dependency                                                                       │
│                                                                                                  │
│   132 │   │   f"Use pip or conda to install {install_name}."                                     │
│   133 │   )                                                                                      │
│   134 │   try:                                                                                   │
│ ❱ 135 │   │   module = importlib.import_module(name)                                             │
│   136 │   except ImportError:                                                                    │
│   137 │   │   if errors == "raise":                                                              │
│   138 │   │   │   raise ImportError(msg)                                                         │
│                                                                                                  │
│ C:\Program                                                                                       │
│ Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\import │
│ lib\__init__.py:126 in import_module                                                             │
│                                                                                                  │
│   123 │   │   │   if character != '.':                                                           │
│   124 │   │   │   │   break                                                                      │
│   125 │   │   │   level += 1                                                                     │
│ ❱ 126 │   return _bootstrap._gcd_import(name[level:], package, level)                            │
│   127                                                                                            │
│   128                                                                                            │
│   129 _RELOADING = {}                                                                            │
│ in _gcd_import:1204                                                                              │
│ in _find_and_load:1176                                                                           │
│ in _find_and_load_unlocked:1140                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'openpyxl'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 db = pd.read_excel('../data/01_raw/Base_clientes_Monopoly.xlsx')                             │
│   2                                                                                              │
│                                                                                                  │
│ C:\Users\Juanjo\Desktop\Monopoly\mono\Lib\site-packages\pandas\io\excel\_base.py:495 in          │
│ read_excel                                                                                       │
│                                                                                                  │
│    492 │                                                                                         │
│    493 │   if not isinstance(io, ExcelFile):                                                     │
│    494 │   │   should_close = True        